In [4]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, r'../../invert')
import mne
import pickle as pkl

from copy import deepcopy
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from invert.simulate import generator
# base_path = "D:/data/flex_ssm/simulation"
base_path = "simulations/"
os.makedirs(base_path, exist_ok=True)
random_seed = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Files

In [8]:
path_fwd = os.path.join(base_path, "../forward_models/fsaverage_biosemi256_Clean_coarse-fwd.fif")
path_info = os.path.join(base_path, "../forward_models/info_biosemi256.pkl")

fwd = mne.read_forward_solution(path_fwd, verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)

fn = path_info
with open(fn, 'rb') as f:
    info = pkl.load(f)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


# Simulate Data

In [9]:
generator_args_baseline = dict(
    use_cov=False,
    return_mask=False,
    batch_repetitions=1,
    batch_size=200,
    return_info=True,
    scale_data=False,
    
    n_sources=2,
    n_orders=(0, 0),
    
    amplitude_range=(1, 1),
    n_timecourses=2000,
    n_timepoints=128,
    inter_source_correlation=0.0,
    
    diffusion_parameter=0.1,
    beta_range=(1, 1),  # Determines the frequency spectrum of each simulted time course (1/f**beta)
    
    snr_range=(0, 0),
    correlation_mode=None,
    noise_color_coeff=0.,
    
    add_forward_error=False,
    forward_error=0.0,
    
    normalize_leadfield=False,
    random_seed=1)

# Figure 1 - ISC with 2 and 3 sources

In [11]:
sim_condition = "figure-1"
fullpath = os.path.join(base_path, f"""sim_{sim_condition}.pkl""")

generator_args = deepcopy(generator_args_baseline)
n_sources_list = (2, 3)
iscs = [0.1, 0.5, 0.9, 0.95, 0.975, 0.99, 1.0]

x_tests = []
y_tests = []
sim_infos = []
for n_sources in n_sources_list:
    print(n_sources)
    generator_args["n_sources"] = n_sources
    for isc in iscs:
        print("\t", isc)
        generator_args["inter_source_correlation"] = isc
        gen_test = generator(fwd, **generator_args)
        x_test, y_test, sim_info = gen_test.__next__()
        x_tests.append(x_test)
        y_tests.append([csr_matrix(yy) for yy in y_test])
        sim_infos.append(sim_info)

x_test = np.concatenate(x_tests, axis=0)
y_test = np.concatenate(y_tests, axis=0)
sim_info = pd.concat(sim_infos)

with open(fullpath, "wb") as f:
    pkl.dump([x_test, y_test, sim_info, generator_args], f)

2
	 0.1
	 0.5
	 0.9
	 0.95
	 0.975
	 0.99
	 1.0
3
	 0.1
	 0.5
	 0.9
	 0.95
	 0.975
	 0.99
	 1.0


FileNotFoundError: [Errno 2] No such file or directory: './simlations/sim_figure-1.pkl'

# Figure 12 - ISC with 3 sources and Cholesky noise at 0.5

In [11]:
sim_condition = "figure-12"
fullpath = os.path.join(base_path, f"""sim_{sim_condition}.pkl""")

generator_args = deepcopy(generator_args_baseline)
generator_args["correlation_mode"] = "cholesky"
generator_args["noise_color_coeff"] = 0.5
n_sources_list = (3,)
iscs = [0.1, 0.5, 0.9, 0.95]#, 0.975, 0.99, 1.0]

x_tests = []
y_tests = []
sim_infos = []
for n_sources in n_sources_list:
    print(n_sources)
    generator_args["n_sources"] = n_sources
    for isc in iscs:
        print("\t", isc)
        generator_args["inter_source_correlation"] = isc
        gen_test = generator(fwd, **generator_args)
        x_test, y_test, sim_info = gen_test.__next__()
        x_tests.append(x_test)
        y_tests.append([csr_matrix(yy) for yy in y_test])
        sim_infos.append(sim_info)

x_test = np.concatenate(x_tests, axis=0)
y_test = np.concatenate(y_tests, axis=0)
sim_info = pd.concat(sim_infos)

with open(fullpath, "wb") as f:
    pkl.dump([x_test, y_test, sim_info, generator_args], f)

3
	 0.1
	 0.5
	 0.9
	 0.95


# Figure 2 - SNR and Noise Color

In [22]:
generator_args = deepcopy(generator_args_baseline)
generator_args["n_orders"] = 0
generator_args["inter_source_correlation"] = 0.5

sim_condition = "figure-2"
fullpath = os.path.join(base_path, f"""sim_{sim_condition}.pkl""")

iscs = (0.5, 0.9)
snr_ranges = [-5, 0, 5]
correlation_modes = [None, "diagonal", "banded", "cholesky", ]  # *noise_covs.values()]
correlation_mode_names = ["White", "Diagonal", "Banded", "Cholesky",]  # "Real Low", "Real Medium", "Real High"]
noise_color_coeff_list = [
    [0,],
    [0, 0.25, 0.5, 0.9],
    [0, 0.25, 0.5, 0.9],
    [0, 0.1, 0.25, 0.5],
    # ["Low",],
    # ["Medium",],
    # ["High",],
    ]


x_tests = []
y_tests = []
sim_infos = []
for correlation_mode, correlation_mode_name, noise_color_coeffs in zip(correlation_modes, correlation_mode_names, noise_color_coeff_list):
    print(correlation_mode_name)
    generator_args["correlation_mode"] = correlation_mode
    for noise_color_coeff in noise_color_coeffs:
        generator_args["noise_color_coeff"] = noise_color_coeff
        for snr_range in snr_ranges:
            print("  ", snr_range, " dB")
            generator_args["snr_range"] = (snr_range, snr_range)
            for isc in iscs:
                print("\t", isc)
                generator_args["inter_source_correlation"] = isc
                gen_test = generator(fwd, **generator_args)
                x_test, y_test, sim_info = gen_test.__next__()
                x_tests.append(x_test)
                y_tests.append([csr_matrix(yy) for yy in y_test])
                sim_infos.append(sim_info)

x_test = np.concatenate(x_tests, axis=0)
y_test = np.concatenate(y_tests, axis=0)
sim_info = pd.concat(sim_infos)

with open(fullpath, "wb") as f:
    pkl.dump([x_test, y_test, sim_info, generator_args], f)

White
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
Diagonal
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
Banded
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
Cholesky
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9
   -5  dB
	 0.5
	 0.9
   0  dB
	 0.5
	 0.9
   5  dB
	 0.5
	 0.9


# Figure 3

In [12]:
generator_args = deepcopy(generator_args_baseline)

sim_condition = "figure-3"
fullpath = os.path.join(base_path, f"""sim_{sim_condition}.pkl""")

time_samples = [8, 16, 32, 64, 128]
inter_source_correlations = (0.5, 0.9, 0.95)

x_tests = []
y_tests = []
sim_infos = []
for inter_source_correlation in inter_source_correlations:
    print(inter_source_correlation)
    generator_args["inter_source_correlation"] = inter_source_correlation
    for n_timepoints in time_samples:
        print("  ", n_timepoints)
        generator_args["n_timepoints"] = n_timepoints
        gen_test = generator(fwd, **generator_args)
        x_test, y_test, sim_info = gen_test.__next__()
        x_tests.append(x_test)
        y_tests.append([csr_matrix(yy) for yy in y_test])
        sim_infos.append(sim_info)

# x_test = np.concatenate(x_tests, axis=0)
# y_test = np.concatenate(y_tests, axis=0)
sim_info = pd.concat(sim_infos)

with open(fullpath, "wb") as f:
    pkl.dump([x_tests, y_tests, sim_info, generator_args], f)

0.5
   8
   16
   32
   64
   128
0.9
   8
   16
   32
   64
   128
0.95
   8
   16
   32
   64
   128
